In [1]:
import numpy as np
import librosa
from keras.models import load_model
from keras.models import Model
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk 
from pydub import AudioSegment
import pygame

# Initialize pygame for audio playback
pygame.mixer.init()

# Function to extract MFCC features from a WAV file
def mfcc_features_extractor_1sec(file):
    y, sr = librosa.load(file, sr=16000)
    y = y[:192000]  # 12 seconds for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])

    # Define window size and hop length
    win_length = int(sr * 1)  # 1 second window
    hop_length = int(win_length / 2)  # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y) - win_length, hop_length):
        mfcc = librosa.feature.mfcc(y=y[i:i + win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

# Function to calculate the closest array
def find_closest_array(intermediate_output, arrays):
    closest_distance = float('inf')  # Initialize closest_distance to positive infinity
    closest_array = None
    closest_array_name = None
    closest_index_within_array = None

    # Define the names of the arrays you want to compare
    array_names_to_compare = ['AL_FALAQ', 'AL_FIL', 'AL_IKHLAS', 'AL_KAFIRUN', 'AL_KAUTHAR',
                              'AL_MASAD', 'AL_MAUN', 'AL_QURAISH', 'AN_NAS', 'AN_NASR']

    # Loop through the array names and calculate distances
    for array_name in array_names_to_compare:
        array = arrays.get(array_name)
        if array is not None:
            distances = np.linalg.norm(array - intermediate_output, axis=1)
            closest_index = np.argmin(distances)
            closest_distance_for_array = distances[closest_index]

            if closest_distance_for_array < closest_distance:
                closest_distance = closest_distance_for_array
                closest_array = array
                closest_array_name = array_name
                closest_index_within_array = closest_index

    return closest_array_name, closest_index_within_array, closest_distance

# Function to process the selected file
def process_selected_file():
    file_name = filedialog.askopenfilename(
        title="Select a WAV file",
        filetypes=[("WAV files", "*.wav")]
    )

    if file_name:
        status_label.config(text="Processing...")
        root.update_idletasks()  # Update the GUI to show the "Processing..." message

        # Extract MFCC features from the selected file
        mfcc_extracted_features_single = mfcc_features_extractor_1sec(file_name)
        mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)
        mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))

        # Load the saved model
        loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

        # Get the intermediate layer output
        intermediate_layer_name = 'dense'
        intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]
        intermediate_layer_model = Model(inputs=loaded_model.input, outputs=loaded_model.layers[intermediate_layer_index].output)
        intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

        # Load the arrays
        arrays = {
            'AL_FALAQ': np.load('al_falaq_mean.npy'),
            'AL_FIL': np.load('al_fil_mean.npy'),
            'AL_IKHLAS': np.load('al_ikhlas_mean.npy'),
            'AL_KAFIRUN': np.load('al_kafirun_mean.npy'),
            'AL_KAUTHAR': np.load('al_kauthar_mean.npy'),
            'AL_MASAD': np.load('al_masad_mean.npy'),
            'AL_MAUN': np.load('al_maun_mean.npy'),
            'AL_QURAISH': np.load('al_quraish_mean.npy'),
            'AN_NAS': np.load('an_nas_mean.npy'),
            'AN_NASR': np.load('an_nasr_mean.npy')
        }

        # Find the closest array
        closest_array_name, closest_index_within_array, closest_distance = find_closest_array(intermediate_output, arrays)

        # Display the result
        result_label.config(text=f"Surah Name: {closest_array_name}\nIndex of Surah: {closest_index_within_array+1}\n{closest_array_name}:{closest_index_within_array+1}")
        status_label.config(text="Completed")

        # Add audio progress bar
        audio_length = librosa.get_duration(filename=file_name)  # Get audio length in seconds
        audio_progress_bar["maximum"] = int(audio_length)
        audio_progress_bar["value"] = 0  # Initialize progress to 0

        # Play the selected audio file
        play_audio(file_name)

    else:
        status_label.config(text="No file selected.")

# Function to play the selected audio file
def play_audio(file_path):
    audio = AudioSegment.from_wav(file_path)
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()

    # Update the progress bar while playing
    while pygame.mixer.music.get_busy():
        current_time = pygame.mixer.music.get_pos() // 1000  # Convert milliseconds to seconds
        audio_progress_bar["value"] = current_time
        root.update_idletasks()  # Update the GUI

# Create the tkinter window
root = tk.Tk()
root.title("Deep Acoustic Modelling for Quranic Recitation")
root.geometry("1024x768")  # Set window size to 1024x768

# Create a header label
header_label = tk.Label(root, text="Deep Acoustic Modelling for Quranic Recitation", font=("Helvetica", 24))
header_label.pack(pady=80)

# Create a separator between left and right sections
separator = ttk.Separator(root, orient="vertical")
separator.pack(fill="y", side="left")

# Create a left frame for the file selection button and audio progress bar
left_frame = tk.Frame(root, width=960, height=1080, padx=200, pady=20)
left_frame.pack(side="left")

# Create a button to trigger file selection
select_file_button = tk.Button(left_frame, text="Choose WAV", command=process_selected_file)
select_file_button.pack(pady=20)

# Create an audio progress bar
audio_progress_bar = ttk.Progressbar(left_frame, orient="horizontal", length=300, mode="determinate")
audio_progress_bar.pack(pady=20)

# Create a right frame for the result and status
right_frame = tk.Frame(root, width=960, height=1080, padx=200, pady=20)
right_frame.pack(side="right")

# Create a label to display the result
result_label = tk.Label(right_frame, text="", font=("Helvetica", 16))
result_label.pack()

# Create a label to display the status
status_label = tk.Label(right_frame, text="", font=("Helvetica", 14))
status_label.pack()

# Start the tkinter main loop
root.mainloop()

C:\Users\aleem\AppData\Roaming\Python\Python39\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 [==============================] - 0s 411ms/step


C:\Users\aleem\AppData\Local\Temp\ipykernel_18568\3917020190.py:112: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  audio_length = librosa.get_duration(filename=file_name)  # Get audio length in seconds
